### DATA IMPORT

In [4]:
#import the reqired libraries
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import numpy as np
import pandas as pd
import math
import json
import time
import matplotlib.pyplot as plt
import seaborn as sns

In [5]:
from sklearn import *

In [6]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.neighbors import NearestNeighbors

In [7]:
#from sklearn.externals import joblib

In [8]:
import scipy.sparse
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import svds
import warnings; warnings.simplefilter('ignore')
%matplotlib inline

In [9]:
oldDt=pd.read_csv("Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products.csv")
oldDt.head(5)

FileNotFoundError: [Errno 2] No such file or directory: 'Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products.csv'

In [ ]:
products={
    'B00IOY8XWQ':'Kindle Voyage 4GB Wifi',
    'B00IOYAM4I':'Kindle Voyage 4GB Wifi+Cellular',
    'B00QFQRELG':'Powerfast Charger',
    'B00REQKWGA':'Kindle + Charging Cover Wifi',
    'B00VINDBJK':'Kindle + Charging Cover Wifi',
    'B00ZV9PXP2':'Kindle 8 Wifi',
    'B010CEHQTG':'Bluetooth Speaker',
    'B017JG41PC':'Kindle Wifi',
    'B0189XYY0Q':'Tablet 10 Wifi 8GB',
    'B018Y224PY':'Tablet with Alexa 16GB',
    'B018Y225IA':'Kindle 16GB Wifi',
    'B018Y22BI4':'Tablet Wifi 16gb',
    'B018Y22C2Y':'Kids Tablet',
    'B018Y23MNM':'Kids Tablet',
    'B01ACEKAJY':'Tablet 8 Wifi 32GB',
    'B01AHB9CN2':'Tablet 8 Wifi 16GB',
    'B01AHB9C1E':'Tablet with Alexa 32GB',
    'B01AHB9CYG':'TV',
    'B01AHBBG04':'Tablet 8 Wifi 16GB',
    'B01AHBDCKQ':'Tablet 8 Wifi 32GB',
    'B01BH83OOM':'Bluetooth Speaker',
    'B01J24C0TI':'Bluetooth Speaker',
    'B01N32NCPM':'TV',
    'B06XB29FPF':'Echo-Plus'
}
oldDt.asins=oldDt.asins.replace(products)
oldDt.head()
    

In [ ]:
dt = oldDt[['reviews.username','asins','reviews.rating']]
dt

In [ ]:
dt.info()

In [ ]:
dt.shape

In [ ]:
print('Lowest rating is: %d' %(dt['reviews.rating'].min()))
print('Highest rating is: %d' %(dt['reviews.rating'].max()))

In [ ]:
dt.isnull().sum()

In [ ]:
with sns.axes_style('white'):
    g = sns.catplot(x="reviews.rating", data=dt, aspect=2.0, kind='count')
    g.set_axis_labels("Ratings", "Total number of ratings")

We can see here that the products get mostly 5 ratings

In [ ]:
# Number of unique user id  in the data
print('Number of unique users in Raw data = ', dt['reviews.username'].nunique())
# Number of unique product id  in the data
print('Number of unique product in Raw data = ', dt['asins'].nunique())

In [ ]:
unique_products = dt['asins'].unique()
print(unique_products)

In [ ]:
most_rated=dt.groupby('reviews.username').size().sort_values(ascending=False)[:10]
print('Top 10 users based on ratings: \n',most_rated)

In [ ]:
counts=dt['reviews.username'].value_counts()
dt1=dt[dt['reviews.username'].isin(counts[counts>5].index)]
dt1

In [ ]:
dt1.describe(include='object')

In [ ]:
print('Number of users who have rated 3 or more items =', len(dt1))
print('Number of unique users in the final data = ', dt1['reviews.username'].nunique())
print('Number of unique products in the final data = ', dt1['asins'].nunique())

In [ ]:
dt1_agg = dt1.groupby(['reviews.username', 'asins'])['reviews.rating'].mean().unstack(fill_value=0)
dt1_agg = dt1_agg.round(decimals=1)  # Round to 1 decimal places

userRates= dt1_agg.rename_axis(None, axis=1)
userRates.shape

In [ ]:
userRates

In [ ]:
#Split the data randomnly into train and test datasets into 70:30 ratio
trainDt, testDt = train_test_split(dt1, test_size = 0.3, random_state=0)
trainDt.head()
testDt.head()

In [ ]:
print('Shape of training data: ',trainDt.shape)
print('Shape of testing data: ',testDt.shape)

In [ ]:
#Count of user_id for each unique product as recommendation score 
trainDt_grouped = trainDt.groupby('asins').agg({'reviews.username': 'count'}).reset_index()
trainDt_grouped.rename(columns = {'reviews.username': 'score'},inplace=True)
trainDt_grouped=trainDt_grouped.sort_values(by='score',ascending=False)
trainDt_grouped.shape
trainDt_grouped

In [ ]:
#Sort the products on recommendation score 
trainDt_sort = trainDt_grouped.sort_values(['score', 'asins'], ascending = [0,1]) 
      
#Generate a recommendation rank based upon score 
trainDt_sort['rank'] = trainDt_sort['score'].rank(ascending=0, method='first') 
          
#Get the top 10 recommendations 
popRec= trainDt_sort.head() 
popRec

the product ranked first is the most popular product.

In [ ]:
# Use popularity based recommender model to make predictions
def recommend(user_id):     
    userRec = popRec
    
    #Add user_id column for which the recommendations are being generated 
    userRec['userID'] =user_id  
    
    #Bring user_id column to the front 
    cols = userRec.columns.tolist() 
    cols = cols[-1:] + cols[:-1] 
    userRec = userRec[cols] 
          
    return userRec

In [ ]:
find_recom = [10,100,150]   # This list is user choice.
for i in find_recom:
    print("The list of recommendations for the userId: %d\n" %(i))
    print(recommend(i))    
    print("\n") 

Since, it is a Popularity recommender model, so, all the three users are given the same recommendations. Here, we predict the products based on the popularity. It is not personalized to particular user. It is a non-personalized recommender system.

#### collaborative filtering

In [ ]:
dt_CF = pd.concat([trainDt, testDt]).reset_index()
dt_CF.head()

In [ ]:
import pandas as pd

pivotDt = round(pd.pivot_table(dt_CF, index='reviews.username', columns='asins', values='reviews.rating', fill_value=0),1)
pivotDt


In [ ]:
pivotDt.shape

In [ ]:
#define user index from 0 to 10
pivotDt['user_index'] = np.arange(0, pivotDt.shape[0], 1)
pivotDt

In [ ]:
pivotDt.set_index(['user_index'], inplace=True)

# Actual ratings given by users
pivotDt

In [ ]:
import scipy.sparse as sp

is_sparse = sp.issparse(pivotDt)

if is_sparse:
    print("The DataFrame is a sparse matrix.")
else:
    print("The DataFrame is not a sparse matrix.")

##### sparse matrix=number of zero in data frame is 2/3 more than number of nonzeros. Since it is not a sparse matrix, svd (singular value decomposition will be applied using numpy library.

In [ ]:
# Convert DataFrame to NumPy array
pvArray = pivotDt.values

# Perform singular value decomposition (a factorization of a real or complex matrix)
U, sigma, Vt = np.linalg.svd(pvArray, full_matrices=False)

In [ ]:
print('Left singular matrix: \n',U)

In [ ]:
print('Sigma: \n', sigma)

In [ ]:
sigma = np.diag(sigma)
print('Diagonal matrix: \n',sigma)

In [ ]:
print('Right singular matrix: \n',Vt)

In [ ]:
#Predicted ratings
predict_all = np.dot(np.dot(U, sigma), Vt)  #The dot product of these matrices approximates the original ratings matrix

# Convert predicted ratings to dataframe
predsDt = pd.DataFrame(predict_all, columns = pivotDt.columns)
predsDt.head()

In [ ]:
# Recommend the items with the highest predicted ratings

def recommend_items(userID, pivotDt, predsDt, num_recommendations):
    # index starts at 0  
    user_idx = userID-1 
    
    # Get and sort the user's ratings
    sorted_user_ratings = pivotDt.iloc[user_idx].sort_values(ascending=False)
    
    #sorted_user_ratings
    sorted_user_predictions = predsDt.iloc[user_idx].sort_values(ascending=False)
    
    #sorted_user_predictions
    temp = pd.concat([sorted_user_ratings, sorted_user_predictions], axis=1)
    temp.index.name = 'Recommended Items'
    temp.columns = ['user_ratings', 'user_predictions']
    temp = temp.loc[temp.user_ratings == 0]   
    temp = temp.sort_values('user_predictions', ascending=False)
    
    print('\nBelow are the recommended items for user(user_id = {}):\n'.format(userID))
    print(temp.head(num_recommendations))

userID = 7
num_recommendations = 5
recommend_items(userID, pivotDt, predsDt, num_recommendations)   

userID = 8
num_recommendations = 5
recommend_items(userID, pivotDt, predsDt, num_recommendations)   

In [ ]:
userRates.head()

In [ ]:
predsDt.head()

In [ ]:
meanRate=userRates.mean()

In [ ]:
meanPred=predsDt.mean()

In [ ]:
rmse_df = pd.concat([meanRate, meanPred], axis=1)
rmse_df.columns = ['Avg_actual_ratings', 'Avg_predicted_ratings']
print(rmse_df.shape)

rmse_df['item_index'] = np.arange(0, rmse_df.shape[0], 1)
rmse_df

In [ ]:
#RMSE=Root Mean Square Error (measure differences between values predicted by a model or an estimator )
from IPython.display import Image
Image("RMSE1.jpg")

In [ ]:
#apply formula
RMSE = round((((rmse_df.Avg_predicted_ratings - rmse_df.Avg_actual_ratings) ** 2).mean()) ** 0.5, 2)
print('\nRMSE SVD Model = {} \n'.format(RMSE))

## GUI



In [ ]:
import pandas as pd
import tkinter as tk

# Function to handle the button click event
def recommend_products():
    # Get user input values
    user_id = int(user_id_var.get())
    num_recommendations = num_recommendations_input.get()
    
    # Perform product recommendation logic
    # Replace this with your actual recommendation algorithm
    recommended_items = recommend_items(user_id, pivotDt, predsDt, num_recommendations)
    
    # Display the recommended products
    output_text.configure(state='normal')
    output_text.delete('1.0', tk.END)
    output_text.insert(tk.END, recommended_items)
    output_text.configure(state='disabled')
    output_text.tag_configure('center', justify='center')
    output_text.tag_add('center', '1.0', 'end')

# Function for product recommendation
def recommend_items(userID, pivotDt, predsDt, num_recommendations):
    user_idx = userID - 1
    sorted_user_ratings = pivotDt.iloc[user_idx].sort_values(ascending=False)
    sorted_user_predictions = predsDt.iloc[user_idx].sort_values(ascending=False)
    temp = pd.concat([sorted_user_ratings, sorted_user_predictions], axis=1)
    temp.index.name = 'Recommended Items'
    temp.columns = ['user_ratings', 'user_predictions']
    temp = temp.loc[temp.user_ratings == 0]
    temp = temp.sort_values('user_predictions', ascending=False)
    return temp.head(num_recommendations).to_string()

# Create the main window
window = tk.Tk()
window.title("Product Recommendation")
window.configure(bg="PeachPuff4")

# Load your data and define pivotDt and predsDt here

# Create main title widget
main_title_label = tk.Label(window, text='Product Recommendation', font=('Cooper Black', 16), bg='PeachPuff4', fg="beige")

# Create input widgets
user_id_label = tk.Label(window, text='User ID:', bg="PeachPuff4", fg="beige")
user_id_var = tk.StringVar(window)
user_id_var.set('1')  # Set initial value
user_id_input = tk.OptionMenu(window, user_id_var, *[str(i) for i in range(1, 33)], command=lambda x: user_id_var.set(x))
num_recommendations_label = tk.Label(window, text='Number of Recommendations:', bg="PeachPuff4", fg="beige")
num_recommendations_input = tk.Scale(window, from_=1, to=10, orient=tk.HORIZONTAL, bg="seashell2")
button = tk.Button(window, text='Recommend', command=recommend_products, font=('Segoe Script', 9), bg="beige")

# Create output widget
output_text = tk.Text(window, height=20, width=70, font=('Times', 11), fg='gray', bg="seashell2")
output_text.configure(state='disabled')

# Pack the widgets
main_title_label.pack(pady=10)
user_id_label.pack()
user_id_input.pack(pady=10)
num_recommendations_label.pack()
num_recommendations_input.pack(pady=10)
button.pack(pady=7)
output_text.pack(expand=True, fill=tk.BOTH)  # Expand the output box to fill available space

# Auto-fit window to its contents
window.update()
window_width = window.winfo_width()
window_height = window.winfo_height()
window.minsize(window_width, window_height)

# Start the main loop
window.mainloop()